In [5]:

# Connetced yourself Cloud Storage
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

import os
os.chdir("/content/gdrive/My Drive/")
os.listdir()

Mounted at /content/gdrive/


['Book.xls',
 'Ch1_1_2.html',
 'profiles_settings.xml',
 '.gitignore',
 'misc.xml',
 'modules.xml',
 'Ch01.iml',
 'workspace.xml',
 'Ch1_6.py',
 'inbound9214322047452905332.jpg',
 'S__41869316.jpg',
 'S__6094934.jpg',
 '20231006_130245.jpg',
 '「英語配音競賽_動物方程市_李宣毅,林湘儀,陳宏駿」.pdf',
 'All_photo_112_萬芳高中-1_01-S301_30111_30111-2 (1).jpg',
 '112_student.csv',
 'Colab Notebooks',
 '112_2_LATIA',
 '112_students_tf.csv',
 '112_ab111_S.csv',
 '112_ab111_C.csv',
 '20240327_233001.jpg',
 '112_ab111_S.gsheet',
 '112_ab111_C.gsheet',
 'Ch1-5.zip.zip 的副本.zip 的副本.zip',
 'TaipeiSansTCBeta-Regular.ttf',
 'Steps 學習單前測.gdoc',
 '「全民國防教育 軍事訓練課程 —防衛動員」_李宣毅.gdoc',
 '20240428_152818 (1).jpg',
 '20240428_152818.jpg',
 'Work Presentation.mp4',
 'Decent Work and Economic Growth_李宣毅_學習科學116.mp4',
 'Work Presentation.pptx',
 'Decent Work and Economic Growth_李宣毅_學習科學116.pptx',
 '3分鐘英語簡報競賽',
 'cpbl.csv',
 'plg.csv',
 'plg_scraper',
 'plg_project',
 'myproject']

In [6]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.4Cy1k6iQXW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.WX9CyPIXxR/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.FmE3npoh0b/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import logging

# Configure logging module
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants definition
URL = 'https://pleagueofficial.com/stat-player'
OUTPUT_FILE = 'plg.csv'
WAIT_TIME = 10

def scrape_plg_data(url=URL, output_file=OUTPUT_FILE, wait_time=WAIT_TIME):
    try:
        # Create Chrome WebDriver with headless mode and other options
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')

        # Initialize Chrome WebDriver, passing the path of Chrome driver and options
        with webdriver.Chrome(options=chrome_options) as driver:
            # Load the target webpage
            driver.get(url)
            logging.info('Web page loaded successfully.')

            # Wait for the webpage to load completely
            driver.implicitly_wait(wait_time)

            # Get the webpage content
            html = driver.page_source

        # Parse HTML using Beautiful Soup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the table element
        table = soup.find('table')

        if table:
            # Use pandas read_html to parse the table
            dfs = pd.read_html(str(table), header=0, keep_default_na=False)

            if dfs:
                # Select the first table
                selected_table = dfs[0]

                # Save data as a CSV file
                selected_table.to_csv(output_file, index=False)
                logging.info(f'Data saved to {output_file}.')

                # Return the selected table
                return selected_table
            else:
                logging.error('No tables found in the HTML.')
        else:
            logging.error('Table element not found in the HTML.')

    except TimeoutException:
        logging.error('Timeout error occurred while loading the web page.')
    except NoSuchElementException:
        logging.error('Element not found error occurred while parsing the HTML.')
    except Exception as e:
        logging.error(f'An error occurred: {str(e)}')

    return None

if __name__ == "__main__":
    scraped_data = scrape_plg_data()
    if scraped_data is not None:
        print(scraped_data)


       球員  背號       球隊  出賽次數 時間 (分)  兩分命中  兩分出手     兩分%  三分命中  三分出手  ...  \
0     吳永盛   6   臺北富邦勇士    23   9:47  0.52  1.48  35.29%  0.48  1.65  ...   
1     李學林   9  桃園璞園領航猿    14   9:28  0.29  0.64  44.44%  0.00  0.07  ...   
2     李威廷   4     新北國王    16    9:1  0.63  1.75  35.71%  0.88  2.25  ...   
3     張文平  10   臺北富邦勇士    16   8:56  0.50  1.13  44.44%  0.69  2.38  ...   
4    陳范柏彥   1   臺北富邦勇士    23   8:50  0.57  1.83  30.95%  0.74  2.22  ...   
..    ...  ..      ...   ...    ...   ...   ...     ...   ...   ...  ...   
122   陳俊男   3     新北國王    21  10:50  0.43  1.62  26.47%  0.29  1.00  ...   
123   呂奇旻  23  新竹御頂攻城獅     3   10:5  0.00  0.67      0%  0.67  1.67  ...   
124   施顏宗   1  新竹御頂攻城獅    23  10:35  0.39  0.96  40.91%  0.22  1.26  ...   
125   張傑瑋   0  新竹御頂攻城獅    22  10:27  1.36  3.77  36.14%  0.14  0.82  ...   
126    田浩   9  新竹御頂攻城獅    19  10:26  0.74  2.32  31.82%  0.58  1.32  ...   

        罰球%    得分    攻板    防板    籃板    助攻    抄截    阻攻    失誤    犯規  
0    42.86%  2.61  